# Loading a network

In this part, we are going to learn to create or load a network with pycaffe.

In [1]:
"""Initialization (see "00 Basic solver usage")."""
import os

import caffe
CAFFE_ROOT="/caffe"
os.chdir(CAFFE_ROOT) # change the current directory to the caffe root, to help
                     # with the relative paths
USE_GPU = True
if USE_GPU:
    caffe.set_device(0)
    caffe.set_mode_gpu()
else:
    caffe.set_mode_cpu()

## Loading a network.

Load the network and set it to the testing phase. To set it to testing phase,
use `caffe.TRAIN`.

In [2]:
network_path = "examples/mnist/lenet_train_test.prototxt"
loaded_network = caffe.Net(network_path, caffe.TEST) # caffe.TEST for testing

## Loading a pretrained network

The network is initialized with the weights in the weights file.

In [3]:
weights_path = "examples/mnist/lenet_iter_10000.caffemodel"
loaded_network = caffe.Net(network_path, weights_path, caffe.TRAIN)

## Creating a network in Python.

In [4]:
from caffe import layers

"""Network specification"""
net = caffe.NetSpec()

### Adding a layer

- `name`: is the first output name (e.g. `net.data`)
- `type`: is the variable type (e.g. `layers.Convolution`)
- `top`: is the return value(s) (e.g. `net.conv1`, or the tuple `(net.data, net.label)`). However, if you have more than one top, you should specify the `ntop` parameter.
- `bottom`: first (positional) argument(s)
- layer parameters: anything that would go in `<layer>_param`, add them as keyword argument. If you need a sub-scope (as for `weight_filler`), use a python dictionary.

#### Data layer

Two outputs (net.data and net.label) are the `top` in the prototxt.
`backend` is specified in the caffe protobuf definition as a DB field, which then becomes

    enum DB {
      LEVELDB = 0;
      LMDB = 1;
    }
    
So the `backend=1` means `LMDB`.

This line is equivalent to the prototxt:

```
layer {
  name: "data"
  type: "Data"
  top: "data"
  top: "label"
  data_param {
    source: "examples/mnist/mnist_train_lmdb"
    batch_size: 64
    backend: LMDB
  }
}
```

In [5]:
# Parameters
batch_size = 64
input_file = "examples/mnist/mnist_train_lmdb"
net.data, net.label = layers.Data(batch_size=batch_size, source=input_file, ntop=2, backend=1)

#### Convolution layer

Here, we have parameters with sub-scopes (`weight_filler`), so we define them with a Python dictionnary.

The statement is equivalent to the prototxt:

```
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 20
    kernel_size: 5
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
```

In [6]:
net.conv1 = layers.Convolution(net.data, num_output=20, kernel_size=5,
                               weight_filler={"type": "xavier"},
                               bias_filler={"type": "constant"})

#### Fully connected layer

Here, we specify a learning rate parameter with the `param` keyword argument

Equivalent prototxt:

```
layer {
  name: "ip2"
  type: "InnerProduct"
  bottom: "conv1"
  top: "ip2"
  param {
    lr_mult: 2
  }
  inner_product_param {
    num_output: 10
  }
}
```

In [7]:
net.ip2 = layers.InnerProduct(net.conv1, param={"lr_mult": 2}, num_output=10)

#### Loss layer

This layer has two inputs, so two positional arguments.

Equivalent prototxt:
    
```   
layer {
  name: "loss"
  type: "SoftMaxWithLoss"
  bottom: "conv1"
  bottom: "label"
  top: "loss"
}
```

In [8]:
net.loss = layers.SoftmaxWithLoss(net.ip2, net.label)

Conversion to a prototxt definiton.

In [9]:
proto = net.to_proto()
print(proto)

layer {
  name: "data"
  type: "Data"
  top: "data"
  top: "label"
  data_param {
    source: "examples/mnist/mnist_train_lmdb"
    batch_size: 64
    backend: LMDB
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 20
    kernel_size: 5
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "ip2"
  type: "InnerProduct"
  bottom: "conv1"
  top: "ip2"
  param {
    lr_mult: 2
  }
  inner_product_param {
    num_output: 10
  }
}
layer {
  name: "loss"
  type: "SoftmaxWithLoss"
  bottom: "ip2"
  bottom: "label"
  top: "loss"
}



Serialized into a string.

In [10]:
str_proto = str(proto)

Write the prototxt definition to a file.

In [11]:
output_file = "my_net.prototxt" # CAFFE_ROOT/output_file
with open(output_file, "w") as f:
    f.write(str_proto)

Load the file.

In [12]:
my_net = caffe.Net(output_file, caffe.TRAIN)

Now that we have a network, what can we do with it? This is the next part of the tutorial.